In [1]:
from pathlib import Path
import sys  
import os
import pandas as pd 
from datetime import datetime

import numpy as np
from operator import itemgetter

In [2]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "libs")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from Utils.load_URM import load_URM
from Utils.load_ICM import load_ICM
from libs.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_all = load_URM("../data/data_train.csv")
ICM_all = load_ICM("../data/data_ICM_metadata.csv")

In [5]:
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
ICM_train, ICM_validation = split_train_in_two_percentage_global_sample(ICM_all, train_percentage = 0.80)

In [6]:
from libs.Evaluation.Evaluator import EvaluatorHoldout
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 183 ( 0.5%) Users that have less than 1 test interactions


In [7]:
from libs.Recommenders.KNN.ItemKNNCBFCFSimilarityHybridRecommender import ItemKNNCBFCFSimilarityHybridRecommender

In [8]:
results = pd.DataFrame()

In [9]:
save_path = "result_experiments/ItemKNNCBFCFSimilarityHybridRecommender/"

In [10]:
if os.path.exists(save_path+"results.csv"):
    results = pd.read_csv(save_path+"results.csv")
else:
    os.makedirs(save_path)

In [13]:
results['MAP'].max()

0.0260388038779782

In [14]:
max_MAP = results['MAP'].max()

In [ ]:
for topK_hybrid in [300, 400]:
    for alpha in np.linspace(0.05, 0.95, 5):
        for topK_knncf in [50, 100]:
            for shrink_knncf in [10, 25, 50]:
                for feature_weighting_knncf in ["TF-IDF"]:
                    for topK_knncbf in [350, 600]:
                        for shrink_knncbf in [150, 300]:
                            for feature_weighting_knncbf in ["none"]:
        
                                current_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                                params = {
                                    "alpha": alpha, 
                                    "topK_hubrid": topK_hybrid,
                                    "topK_knncf": topK_knncf,
                                    "shrink_knncf": shrink_knncf,
                                    "feature_weighting_knncf": feature_weighting_knncf,
                                    "topK_knncbf": topK_knncbf,
                                    "shrink_knncbf": shrink_knncbf,
                                    "feature_weighting_knncbf": feature_weighting_knncbf,
                                    "time": current_timestamp
                                }
                                
                                model_knn = ItemKNNCBFCFSimilarityHybridRecommender(
                                    alpha = alpha,
                                    topK_hybrid=topK_hybrid,
                                    URM_train = URM_train,
                                    ICM_train = ICM_train,
                                    topK_knncf=topK_knncf,
                                    shrink_knncf=shrink_knncf,
                                    feature_weighting_knncf=feature_weighting_knncf,
                                    topK_knncbf=topK_knncbf,
                                    shrink_knncbf=shrink_knncbf,
                                    feature_weighting_knncbf=feature_weighting_knncbf,
                                )
        
                                metrics_df, _ = evaluator_validation.evaluateRecommender(model_knn)
                                if max_MAP < metrics_df["MAP"][10]:
                                    max_MAP = metrics_df["MAP"][10]
                                print(', '.join(f"{key}: {value}" for key, value in params.items()), "MAP: ", metrics_df["MAP"][10], "max MAP: ", max_MAP)
                                      
                                params_df = pd.DataFrame([params])
                                iteration_results = pd.concat([params_df, metrics_df.reset_index()], axis=1)
                                results = pd.concat([results, iteration_results], ignore_index=True)
                                results.to_csv(save_path+"results.csv", index=False)

Similarity column 38121 (100.0%), 4690.40 column/sec. Elapsed time 8.13 sec
ItemKNNCBFRecommender: ICM Detected 36 ( 0.1%) items with no features.
Similarity column 38121 (100.0%), 1473.45 column/sec. Elapsed time 25.87 sec
EvaluatorHoldout: Processed 35553 (100.0%) in 17.05 sec. Users per second: 2085
alpha: 0.05, topK_hubrid: 300, topK_knncf: 50, shrink_knncf: 10, feature_weighting_knncf: TF-IDF, topK_knncbf: 350, shrink_knncbf: 150, feature_weighting_knncbf: none, time: 2024-12-01 23:00:19 MAP:  0.012283261207613902 max MAP:  0.0260388038779782
Similarity column 38121 (100.0%), 4708.90 column/sec. Elapsed time 8.10 sec
ItemKNNCBFRecommender: ICM Detected 36 ( 0.1%) items with no features.
Similarity column 38121 (100.0%), 1490.21 column/sec. Elapsed time 25.58 sec
EvaluatorHoldout: Processed 35553 (100.0%) in 16.70 sec. Users per second: 2129
alpha: 0.05, topK_hubrid: 300, topK_knncf: 50, shrink_knncf: 10, feature_weighting_knncf: TF-IDF, topK_knncbf: 350, shrink_knncbf: 300, featur

In [36]:
results.sort_values('MAP', ascending=False).iloc[:, : 15].head(25)

,alpha,topK_hubrid,topK_knncf,shrink_knncf,feature_weighting_knncf,topK_knncbf,shrink_knncbf,feature_weighting_knncbf,time,cutoff,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN
1401,0.15,25,25,50,TF-IDF,300,150,none,2024-11-29 08:04:00,10,0.053248,0.081631,0.070687,0.026039,0.039483
1725,0.15,25,50,50,TF-IDF,300,150,none,2024-11-29 12:36:58,10,0.053363,0.081685,0.070617,0.025999,0.039371
2049,0.15,25,100,50,TF-IDF,300,150,none,2024-11-29 16:48:02,10,0.053332,0.081428,0.070238,0.02593,0.039202
1365,0.15,25,25,50,BM25,300,150,none,2024-11-29 07:38:38,10,0.052978,0.081164,0.070277,0.025922,0.039336
1689,0.15,25,50,50,BM25,300,150,none,2024-11-29 11:27:12,10,0.053062,0.081162,0.07014,0.025885,0.03919
2013,0.15,25,100,50,BM25,300,150,none,2024-11-29 16:21:32,10,0.053116,0.08106,0.069918,0.025821,0.039009
1509,0.15,25,25,100,TF-IDF,300,150,none,2024-11-29 09:20:06,10,0.052722,0.080873,0.070053,0.025736,0.039058
1833,0.15,25,50,100,TF-IDF,300,150,none,2024-11-29 13:56:29,10,0.052711,0.080737,0.069851,0.025693,0.038947
2157,0.15,25,100,100,TF-IDF,300,150,none,2024-11-29 22:39:25,10,0.052714,0.080512,0.069478,0.025626,0.038762
1473,0.15,25,25,100,BM25,300,150,none,2024-11-29 08:54:45,10,0.052163,0.079973,0.069232,0.0255,0.038722


In [35]:
results

,alpha,topK_hubrid,topK_knncf,shrink_knncf,feature_weighting_knncf,topK_knncbf,shrink_knncbf,feature_weighting_knncbf,time,cutoff,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN
0,0.05,25,25,50,none,25,50,none,2024-11-28 12:45:44,10,0.028848,0.043236,0.036654,0.013427,0.020238
1,0.05,25,25,50,none,25,50,BM25,2024-11-28 12:46:27,10,0.029062,0.044,0.037474,0.013514,0.020398
2,0.05,25,25,50,none,25,50,TF-IDF,2024-11-28 12:47:11,10,0.032764,0.04966,0.04236,0.015039,0.022692
3,0.05,25,25,50,none,25,100,none,2024-11-28 12:47:54,10,0.028542,0.042805,0.036427,0.013371,0.020174
4,0.05,25,25,50,none,25,100,BM25,2024-11-28 12:48:37,10,0.031555,0.047801,0.040747,0.014578,0.021969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2164,0.15,25,100,150,none,25,100,BM25,2024-11-29 22:44:29,10,0.032432,0.049211,0.04196,0.015062,0.022694
2165,0.15,25,100,150,none,25,100,TF-IDF,2024-11-29 22:45:13,10,0.036623,0.055464,0.0474,0.016738,0.02526
2166,0.15,25,100,150,none,25,150,none,2024-11-29 22:45:56,10,0.030689,0.046684,0.040087,0.014525,0.022094
2167,0.15,25,100,150,none,25,150,BM25,2024-11-29 22:46:40,10,0.034166,0.05167,0.044038,0.015792,0.023806
